In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:26
🔁 Restarting kernel...


Установка необходимых пакетов

In [ ]:
! conda install -y -c bioconda fastqc

In [ ]:
!conda install -c bioconda samtools

In [ ]:
! conda install -y -c bioconda hisat2

In [ ]:
!ls drive/MyDrive/ngs_rna-seq_data

B14.5.fq.gz  B17.5.fq.gz  B34.fq.gz    C15.5.fq.gz  C20.fq.gz
B15.5.fq.gz  B20.fq.gz	  C14.5.fq.gz  C17.5.fq.gz  C34.fq.gz


Загрузка файлов с Ensembl

In [ ]:
!wget http://ftp.ensembl.org/pub/release-107/fasta/mus_musculus/dna/Mus_musculus.GRCm39.dna.chromosome.19.fa.gz
#19 chromosome sequence

In [ ]:
!wget http://ftp.ensembl.org/pub/release-107/gtf/mus_musculus/Mus_musculus.GRCm39.107.chr.gtf.gz
#annotation

In [ ]:
!mkdir genome-annotation
!mv Mus_musculus.GRCm39.107.chr.gtf.gz genome-annotation
!mv Mus_musculus.GRCm39.dna.chromosome.19.fa.gz genome-annotation
!gunzip ./genome-annotation/*

Анализ входных ридов в FastQC

In [ ]:
! mkdir fastqc-reports

In [ ]:
!fastqc -o fastqc-reports drive/MyDrive/ngs_rna-seq_data/*

In [ ]:
! conda install -c bioconda -c conda-forge multiqc

In [ ]:
!multiqc fastqc-reports/


  /// ]8;id=616436;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.13

|           multiqc | Search path : /content/fastqc-reports
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 20/20  
|            fastqc | Found 10 reports
|           multiqc | Compressing plot data
|           multiqc | Report      : multiqc_report.html
|           multiqc | Data        : multiqc_data
|           multiqc | MultiQC complete


Не требовалась обработка ридов после анализа в FastQC. Сразу перешла к картированию.

In [ ]:
%cd ./genome-annotation

/content/genome-annotation


In [ ]:
!grep -P '^19\t' Mus_musculus.GRCm39.107.chr.gtf > chr19.gtf
#filtering only 19 chromosome from annotation file

In [ ]:
#using function to extract exons and splice sites coordinates
!hisat2_extract_splice_sites.py chr19.gtf > chr19.splicesites  
!hisat2_extract_exons.py chr19.gtf > chr19.exons

Построение индекса по последовательности 19ой хромосомы (сразу с координатами сайтов)

In [ ]:
!hisat2-build --ss chr19.splicesites --exon chr19.exons Mus_musculus.GRCm39.dna.chromosome.19.fa Mus_musculus.GRCm39.dna.chromosome.19

In [ ]:
%cd ../

/content


In [ ]:
!mkdir bam_files #directory for hisat2 output files

Картирование fq файлов с аргументами: --no-softclip - не обрезать риды; --summary - печатать summary;
--no-unal - не печатать риды, которые не выровнялись на геном;
-U - путь к входным ридам;
-x - путь к индексу

In [ ]:
f = 'B34'
index = './genome-annotation/Mus_musculus.GRCm39.dna.chromosome.19'
file_path = './drive/MyDrive/ngs_rna-seq_data'
splice = './genome-annotation/chr19.splicesites'
out_path = './bam_files'

In [ ]:
!hisat2 --summary-file $f\.log --no-softclip --no-unal --known-splicesite-infile $splice -x $index -U $file_path/$f\.fq.gz | samtools view -Sb - > $out_path/$f\.bam

599683 reads; of these:
  599683 (100.00%) were unpaired; of these:
    51800 (8.64%) aligned 0 times
    539811 (90.02%) aligned exactly 1 time
    8072 (1.35%) aligned >1 times
91.36% overall alignment rate


In [ ]:
%%shell
reads='B14.5 B17.5 B15.5 B20 C15.5 C14.5 C17.5 C20 C34'
index='./genome-annotation/Mus_musculus.GRCm39.dna.chromosome.19'
file_path='./drive/MyDrive/ngs_rna-seq_data'
splice='./genome-annotation/chr19.splicesites'
out_path='./bam_files'
for read in ${reads}
do hisat2 --summary-file ${read}.log --no-softclip --no-unal --known-splicesite-infile ${splice}  -x ${index} -U ${file_path}/${read}.fq.gz | samtools view -Sb - > ${out_path}/${read}.bam
done

Образец C20

In [ ]:
! samtools sort -o B20_sorted.bam ./bam_files/B20.bam

In [ ]:
!samtools index B20_sorted.bam

In [ ]:
!samtools view B20_sorted.bam 19:12485000-12490000

HISEQ:129:C2LRCACXX:1:2110:5448:97880	272	19	12479312	0	42M56847N42M50311N17M	*	0	0	TCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTTGTGGGGAG	###DDDDDDDDDDBB@BBDDDDDBB@B??<?B@B<B@B@DD@=<BBDDCCDFEFHHHJJJIGJJJIJIHDGGIJJJIJJIIJIIJIG=GHGHFFFFFFCC@	AS:i:-16	ZS:i:-16	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:101	YT:Z:UU	XS:A:+	NH:i:4
HISEQ:129:C2LRCACXX:1:2110:5448:97880	272	19	12479314	0	42M56845N42M50311N17M	*	0	0	TCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTTGTGGGGAG	###DDDDDDDDDDBB@BBDDDDDBB@B??<?B@B<B@B@DD@=<BBDDCCDFEFHHHJJJIGJJJIJIHDGGIJJJIJJIIJIIJIG=GHGHFFFFFFCC@	AS:i:-16	ZS:i:-16	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:101	YT:Z:UU	XS:A:+	NH:i:4
HISEQ:129:C2LRCACXX:1:2110:5448:97880	272	19	12479316	0	42M56843N42M50311N17M	*	0	0	TCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTTGTGGGGAG	###DDDDDDDDDDBB@BBDDDDDBB@B??<?B@B<B@B@DD@=<BBDDCCDFEFHHHJJJIGJJJIJIHD

7 ридов, 3 выровнялись на весь геном, один хит (по NH tag)

In [ ]:
!samtools view -c B20_sorted.bam

561335


In [ ]:
%%shell
mismatches='0 1 2 3 4 5 6 7 8 9 10 11 12 13 14'
for mismatch in ${mismatches}
do samtools view B20_sorted.bam | grep -cw NM:i:${mismatch}
done

409393
105477
26331
11612
4038
2213
1294
599
268
64
36
1
5
3
1


Сверху перечислено количество ридов, картировавшихся без замен, и с 1-14 заменами

In [ ]:
!samtools view B20_sorted.bam | awk '$6!="101M"{count++};END{print count;}'

140442


19:12485000-12490000 42M56847N42M50311N17M  
19:12485000-12490000 42M56845N42M50311N17M  
19:12485000-12490000 42M56843N42M50311N17M  
19:12485000-12490000 42M56841N42M50311N17M




4 рида картировались на экзон-экзонные границы